This is the data setup for an NFL machine learning model. This file will be for the data ingestion and transformation, and I will have multiple other files for different types of modeling.

# Imports & Housekeeping

In [4]:
# Basic Packages
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime as dt

# Visualizations
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
# Notebook display options
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)

# Data Import

This is not the full raw data. This data was acquired in it's raw form from nfl_data_py api. Transformations and EPA calculations were done in Pycharm because of easier access to classes, functions, etc. At this point, Google Notebooks is a better place for the modeling. Credit: https://github.com/cooperdff/nfl_data_py.

In [7]:
nfl_api = pd.read_csv("../API Data Out/test_2003_to_2023.csv", index_col=0)
schedule = pd.read_csv("../API Data Out/schedule_2003_to_2023.csv", index_col=0)

In [8]:
# We want a df that we will model on without anything that isn't a feature or target
model_df = nfl_api

model_df.sort_values(by='game_id', ascending=False)

,game_id,season,week,team,opponent,score,home,...,ewma_dynamic_window_rushing_defense_team,ewma_dynamic_window_passing_defense_team,ewma_dynamic_window_rushing_offense_opp,ewma_dynamic_window_passing_offense_opp,ewma_dynamic_window_rushing_defense_opp,ewma_dynamic_window_passing_defense_opp,team_id
10849,2023_03_TEN_CLE,2023,3,TEN,CLE,3,0,...,-0.26,0.13,0.01,-0.17,-0.11,-0.23,2023_03_TEN_CLE_A
10848,2023_03_TEN_CLE,2023,3,CLE,TEN,27,1,...,-0.11,-0.23,-0.05,-0.14,-0.26,0.13,2023_03_TEN_CLE_H
10847,2023_03_PIT_LV,2023,3,PIT,LV,23,0,...,-0.05,-0.01,-0.17,0.04,0.03,0.11,2023_03_PIT_LV_A
10846,2023_03_PIT_LV,2023,3,LV,PIT,18,1,...,0.03,0.11,-0.08,-0.05,-0.05,-0.01,2023_03_PIT_LV_H
10845,2023_03_PHI_TB,2023,3,PHI,TB,25,0,...,-0.16,-0.04,-0.16,0.08,-0.15,0.08,2023_03_PHI_TB_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2003_01_BAL_PIT,2003,1,BAL,PIT,15,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003_01_BAL_PIT_A
2,2003_01_ATL_DAL,2003,1,DAL,ATL,13,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2003_01_ATL_DAL_H
3,2003_01_ATL_DAL,2003,1,ATL,DAL,27,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003_01_ATL_DAL_A
1,2003_01_ARI_DET,2003,1,ARI,DET,24,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003_01_ARI_DET_A


Ok, this is our starting df. The target is score_diff - The difference in score between two teams. In this case, the score diff is with respect to the home team. This mean if the diff is positive, the home team won. We will keep this in mind for bringing in other features.

The EPA columns are created by bringing in play by play data, rolling them up into games, and then creating a 10 game rolling average offset by one week so that each EPA that we see is an average of the last 10 games. If the team has not played 10 games in the season, it will take as many games as have been played.

# (Other) Feature Engineering

Adding in a couple of features I think will be valuable to model on

In [9]:
feature_df = schedule
feature_df.columns

Index(['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
       'gametime', 'away_team', 'away_score', 'home_team', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game',
       'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id',
       'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium'],
      dtype='object')

In [10]:
# Was wind involved in the game. I'm setting the threshold at 15mph, but may adjust later
feature_df['windy'] = np.where(schedule.loc[:, 'wind'] > 15, 1, 0)

# Next lets look at "rest differential". A further analysis of this can be seen on my github
feature_df['rest_differential'] = schedule.loc[:, "home_rest"] - schedule.loc[:, "away_rest"]
feature_df

,game_id,season,game_type,week,gameday,weekday,gametime,...,away_coach,home_coach,referee,stadium_id,stadium,windy,rest_differential
1044,2003_01_NYJ_WAS,2003,REG,1,2003-09-04,Thursday,09:00,...,Herm Edwards,Steve Spurrier,Walt Coleman,WAS00,FedExField,0,0
1045,2003_01_NE_BUF,2003,REG,1,2003-09-07,Sunday,13:00,...,Bill Belichick,Gregg Williams,Jeff Triplette,BUF00,Ralph Wilson Stadium,0,0
1046,2003_01_JAX_CAR,2003,REG,1,2003-09-07,Sunday,13:00,...,Jack Del Rio,John Fox,Johnny Grier,CAR00,Ericsson Stadium,0,0
1047,2003_01_DEN_CIN,2003,REG,1,2003-09-07,Sunday,13:00,...,Mike Shanahan,Marvin Lewis,Gerry Austin,CIN00,Paul Brown Stadium,0,0
1048,2003_01_IND_CLE,2003,REG,1,2003-09-07,Sunday,13:00,...,Tony Dungy,Butch Davis,Walt Anderson,CLE00,Cleveland Browns Stadium,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6688,2023_18_ATL_NO,2023,REG,18,2024-01-07,Sunday,13:00,...,Arthur Smith,Dennis Allen,NaN,NOR00,Mercedes-Benz Superdome,0,-1
6689,2023_18_PHI_NYG,2023,REG,18,2024-01-07,Sunday,13:00,...,Nick Sirianni,Brian Daboll,NaN,NYC01,MetLife Stadium,0,0
6690,2023_18_LA_SF,2023,REG,18,2024-01-07,Sunday,13:00,...,Sean McVay,Kyle Shanahan,NaN,SFO01,Levi's Stadium,0,-7
6691,2023_18_JAX_TEN,2023,REG,18,2024-01-07,Sunday,13:00,...,Doug Pederson,Mike Vrabel,NaN,NAS00,Nissan Stadium,0,0


Ok lets reduce the columns to the ones we need and merge with our original df

In [11]:
merge_df = feature_df[['game_id', 'rest_differential', 'windy', 'div_game', 'spread_line', 'total_line', 'game_type', 'location', 'total']]

In [12]:
final_df = pd.merge(model_df, merge_df, on="game_id", how='left')

# Only reg season games
final_df = final_df.loc[final_df.game_type == "REG"]

In [19]:
# Since we're now doing 1 team per row, need to make some adjustments to the values
final_df["implied_points"] = np.where(final_df.home == 1, final_df.total_line/2 + final_df.spread_line/2, final_df.total_line/2 - final_df.spread_line/2)
final_df.spread_line = np.where(final_df.home == 1, final_df.spread_line, final_df.spread_line*-1)
final_df.rest_differential = np.where(final_df.home == 1, final_df.rest_differential, final_df.rest_differential*-1)

In [20]:
final_df.tail(50)

,game_id,season,week,team,opponent,score,home,...,div_game,spread_line,total_line,game_type,location,total,implied_points
10800,2023_02_LV_BUF,2023,2,BUF,LV,38,1,...,0,7.50,47.00,REG,Home,48.00,27.25
10801,2023_02_LV_BUF,2023,2,LV,BUF,10,0,...,0,-7.50,47.00,REG,Home,48.00,19.75
10802,2023_02_MIA_NE,2023,2,NE,MIA,17,1,...,1,-2.00,46.50,REG,Home,41.00,22.25
10803,2023_02_MIA_NE,2023,2,MIA,NE,24,0,...,1,2.00,46.50,REG,Home,41.00,24.25
10804,2023_02_MIN_PHI,2023,2,PHI,MIN,34,1,...,0,6.00,49.00,REG,Home,62.00,27.50
10805,2023_02_MIN_PHI,2023,2,MIN,PHI,28,0,...,0,-6.00,49.00,REG,Home,62.00,21.50
10806,2023_02_NO_CAR,2023,2,CAR,NO,17,1,...,1,-3.00,39.50,REG,Home,37.00,18.25
10807,2023_02_NO_CAR,2023,2,NO,CAR,20,0,...,1,3.00,39.50,REG,Home,37.00,21.25
10808,2023_02_NYG_ARI,2023,2,ARI,NYG,28,1,...,0,-5.00,40.00,REG,Home,59.00,17.50
10809,2023_02_NYG_ARI,2023,2,NYG,ARI,31,0,...,0,5.00,40.00,REG,Home,59.00,22.50


ok lets output back to csv so that we can bring it in the other files to model on without messing with the final result

In [21]:
final_df.to_csv("Data/final_data.csv")

Okay we should probably do some visualizations to see what this data actually is telling us